**Overview**:

This notebook tests the baseline megadetector on the COD10K dataset. It evaluates the detector only, which detects animals correctly only 50% of the time. The notebook doesn't contain an evaluation of the classifiers however, we did test the 3 classifiers provided by Pytorch Wildlife manually, and they performed really bad, almost classifying 1 of 4 images correctly. A mini evaluation report is shown as the output of the second last cell where it shows how many images were classified correctly and how many were not. The average confidence level is also reported.

**Why we didn't evaluate the classifiers:**

This project's pipeline is that the detector first detects an animal and then sends it to the classifier, we have found that the detector fails 50% of the time, which means improving it should be our first priority, and not the classifier.

**Moving Forward**:

We aim to do 2 things moving forward, firstly improve the detector using state of the art COD models such as HighResNet.

Secondly, we were able to make some progress obtaining the LILA Dataset, which is actually the most relevant dataset to this project since its the only one containing camera trap images. We hope to have obtained a relevant subset of the LILA dataset by the next checkpoint and have tested the megadetector on it as well, maybe then we could move towards improving the classifiers of Pytorch Wildlife.  



In [1]:
!pip install PytorchWildlife

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 820.5/820.5 kB 26.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.5/151.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

In [1]:
import numpy as np
from google.colab import drive
import cv2
from PytorchWildlife.models import detection as pw_detection
from PytorchWildlife.models import classification as pw_classification



wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results


wandb: Enter your choice: (30 second timeout) 3


wandb: You chose "Don't visualize my results"
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
detection_model = pw_detection.MegaDetectorV6(version="MDV6-yolov10-e") # Model weights are automatically downloaded.
classification_model = pw_classification.AI4GAmazonRainforest()

Ultralytics 8.3.108 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv10x summary (fused): 215 layers, 31,589,858 parameters, 0 gradients, 169.8 GFLOPs


Downloading: "https://zenodo.org/records/14252214/files/AI4GAmazonDeforestationv2?download=1" to /root/.cache/torch/hub/checkpoints/AI4GAmazonDeforestationv2
100%|██████████| 188M/188M [00:16<00:00, 12.2MB/s]


In [7]:
img = cv2.imread('/content/drive/MyDrive/Images_unzipped/Image/COD10K-CAM-2-Terrestrial-21-Ant-1233.jpg')
result = detection_model.single_image_detection(img)
print(result)


0: 640x640 1 animal, 95.2ms
Speed: 23.2ms preprocess, 95.2ms inference, 112.7ms postprocess per image at shape (1, 3, 640, 640)
{'img_id': 'None', 'detections': Detections(xyxy=array([[     170.26,      90.752,      951.45,      588.08]], dtype=float32), mask=None, confidence=array([    0.93529], dtype=float32), class_id=array([0]), tracker_id=None, data={}), 'labels': ['animal 0.94']}


In [6]:
import zipfile
import os
drive.mount('/content/drive')
image_folder = '/content/drive/MyDrive/Images.zip'
extract_to = '/content/drive/MyDrive/Images_unzipped'
os.makedirs(extract_to, exist_ok=True)

with zipfile.ZipFile(image_folder, 'r') as zip_ref:
    zip_ref.extractall(extract_to)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
from tqdm import tqdm

image_folder = '/content/drive/MyDrive/Images_unzipped/Image'

# Initialize counters and confidence trackers
total_images = 0
total_animals = 0
total_humans = 0
total_vehicles = 0
none_detected = 0
wrong_predictions = 0
confidences = []


for img_file in tqdm(os.listdir(image_folder)):
    print("image_file: ", img_file)
    if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    total_images += 1
    img_path = os.path.join(image_folder, img_file)


    img = cv2.imread(img_path)
    if img is None:
        print(f"Failed to load {img_file}")
        continue




    results = detection_model.single_image_detection(img)

    print(results)

    human_detected = False
    animal_detected = False



    for label in results['labels']:

        if 'person' in label.lower():

            human_detected = True
            conf = float(label.split()[-1])
            confidences.append(conf)

        elif 'animal' in label.lower():

            animal_detected = True
            conf = float(label.split()[-1])
            confidences.append(conf)

        elif 'vehicle' in label.lower():
            total_vehicles += 1

        elif not results['labels']:
            none_detected += 1

    is_human_image = 'Human' in img_file

    if is_human_image:
        if not human_detected:
            wrong_predictions += 1
        else:
            total_humans += 1
    else:
        if not animal_detected:
            wrong_predictions += 1
        else:
            total_animals += 1


Avg_confidence = np.mean(confidences)


print("total_images: ", total_images)
print("total_animals_identified_correctly: ", total_animals)
print("total_humans_identified_correctly: ", total_humans)
print("total_vehicles: ", total_vehicles)
print("None: ", none_detected)

print("wrong predictions: ", wrong_predictions)
print("Average Confidence Level: ", Avg_confidence)





  0%|          | 0/1552 [00:00<?, ?it/s]

image_file:  COD10K-CAM-2-Terrestrial-21-Ant-1233.jpg

0: 640x640 1 animal, 80.1ms
Speed: 6.5ms preprocess, 80.1ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)


  0%|          | 1/1552 [00:00<02:40,  9.67it/s]

{'img_id': 'None', 'detections': Detections(xyxy=array([[     170.26,      90.752,      951.45,      588.08]], dtype=float32), mask=None, confidence=array([    0.93529], dtype=float32), class_id=array([0]), tracker_id=None, data={}), 'labels': ['animal 0.94']}
image_file:  COD10K-CAM-2-Terrestrial-21-Ant-1238.jpg

0: 640x640 1 vehicle, 79.9ms
Speed: 3.4ms preprocess, 79.9ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
{'img_id': 'None', 'detections': Detections(xyxy=array([[    0.56807,           0,      1021.2,         683]], dtype=float32), mask=None, confidence=array([    0.56125], dtype=float32), class_id=array([2]), tracker_id=None, data={}), 'labels': ['vehicle 0.56']}
image_file:  COD10K-CAM-2-Terrestrial-21-Ant-1240.jpg

0: 640x640 (no detections), 73.6ms
Speed: 3.2ms preprocess, 73.6ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)


  0%|          | 3/1552 [00:00<02:26, 10.56it/s]

...

{'img_id': 'None', 'detections': Detections(xyxy=array([[     495.95,      279.57,      876.62,      449.96]], dtype=float32), mask=None, confidence=array([     0.8715], dtype=float32), class_id=array([0]), tracker_id=None, data={}), 'labels': ['animal 0.87']}
image_file:  COD10K-CAM-5-Other-69-Other-5060.jpg

0: 640x640 (no detections), 40.2ms
Speed: 4.9ms preprocess, 40.2ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
{'img_id': 'None', 'detections': Detections(xyxy=array([], shape=(0, 4), dtype=float32), mask=None, confidence=array([], dtype=float32), class_id=array([], dtype=int64), tracker_id=None, data={}), 'labels': []}
image_file:  COD10K-CAM-5-Other-69-Other-5062.jpg

0: 640x640 1 animal, 45.3ms
Speed: 3.9ms preprocess, 45.3ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)


100%|█████████▉| 1551/1552 [01:57<00:00, 10.77it/s]

{'img_id': 'None', 'detections': Detections(xyxy=array([[     343.21,      290.13,       805.5,      644.06]], dtype=float32), mask=None, confidence=array([    0.95807], dtype=float32), class_id=array([0]), tracker_id=None, data={}), 'labels': ['animal 0.96']}
image_file:  COD10K-CAM-5-Other-69-Other-5063.jpg

0: 640x640 1 animal, 44.5ms
Speed: 3.6ms preprocess, 44.5ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)


100%|██████████| 1552/1552 [01:58<00:00, 13.15it/s]

{'img_id': 'None', 'detections': Detections(xyxy=array([[     95.227,      163.15,      877.34,      490.74]], dtype=float32), mask=None, confidence=array([    0.68364], dtype=float32), class_id=array([0]), tracker_id=None, data={}), 'labels': ['animal 0.68']}
total_images:  1552
total_animals:  797
total_humans:  14
total_vehicales:  91
None:  0
wrong predictions:  741
Average Confidence Level:  0.5217691659646168


In [13]:
img_name = "/content/drive/MyDrive/Images_unzipped/Image/COD10K-CAM-2-Terrestrial-34-Human-2010.jpg"

is_human_image = 'Human' in img_name

print(is_human_image)


True
